In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# Обработка данных

In [2]:
data = pd.read_csv('SpotifyFeatures.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232725 entries, 0 to 232724
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   genre             232725 non-null  object 
 1   artist_name       232725 non-null  object 
 2   track_name        232725 non-null  object 
 3   track_id          232725 non-null  object 
 4   popularity        232725 non-null  int64  
 5   acousticness      232725 non-null  float64
 6   danceability      232725 non-null  float64
 7   duration_ms       232725 non-null  int64  
 8   energy            232725 non-null  float64
 9   instrumentalness  232725 non-null  float64
 10  key               232725 non-null  object 
 11  liveness          232725 non-null  float64
 12  loudness          232725 non-null  float64
 13  mode              232725 non-null  object 
 14  speechiness       232725 non-null  float64
 15  tempo             232725 non-null  float64
 16  time_signature    23

## Числовые признаки

Выделим для начала числовые признаки, так как они дополнительной обработки не требуют. Исходя из условия, *popularity* - целевая переменная, поэтому удалю ее из этого списка.

In [23]:
numeric = data.columns[data.dtypes != 'object']
numeric = numeric.drop('popularity')
numeric

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence'],
      dtype='object')

Все признаки будем использовать в модели

## Категориальные признаки

**genre** - жанр песни - поддается обработке, потенциально высокая связь с популярностью

**artist_name** - имя артиста - слишком много уникальных значений, вряд ли влияет на популярность

**track_name** - название трека - слишком много уникальных значений, вряд ли влияет на популярность

**track_id** - уникальный идентификатор трека - слишком много уникальных значений, никак не связано с популярностью

**key** - аккорд - поддается обработке, связь с популярностью должна отсутствовать

**mode** - тональность - поддается обработке, связь с популярностью должна отсутствовать

**time_signature** - размерность - поддается обработке, связь с популярностью должна отсутствовать

Выберем только жанр. Применим для этого признака OneHot кодирование

In [44]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(data[numeric].join(data.genre))
y = data.popularity

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Обучение моделей

Выбрал простую линейную регресиию без регуляризации и лес решающих деревьев

In [67]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge

rfr = RandomForestRegressor(random_state=0, n_jobs=-1)
lr = LinearRegression(normalize=True, n_jobs=-1)

lr.fit(X_train, y_train)
rfr.fit(X_train, y_train)

RandomForestRegressor(n_jobs=-1, random_state=0)

# Валидация моделей

Пусть результатом, от которого мы будем отталкиваться при валидации моделей, будет значение средней ошибки, когда каждой песне мы ставим в качестве ответа 50 - среднюю возможную оценку.

In [80]:
baseline = [50] * y_test.size
mean_absolute_error(y_test, baseline)

15.913461951617755

In [70]:
print(mean_absolute_error(y_test, lr.predict(X_test)))
print(mean_absolute_error(y_test, rfr.predict(X_test)))

7.228431010284181
7.053317505161305


Как можно видеть, линейная регрессия немного уступает лесу деревьев, однако стоит заметить, что обучается она гораздо быстрее